# Prerrequisites

Installing Spark

---



In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip -q install findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()

Starting Spark Session and print the version


---


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# create the session
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.2.0'

Creating tunnel</br>
**To Check the Spark UI, open the URL printed by running the above command : https://######/jobs/, /SQL/**


In [5]:
 from google.colab.output import eval_js
 print(eval_js("google.colab.kernel.proxyPort(4040)") + "jobs/")

https://4ogobv14ej2-496ff2e9c6d22116-4040-colab.googleusercontent.com/jobs/


# Descargar Datasets

In [6]:
!mkdir -p /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/frankenstein.txt -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/el_quijote.txt -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/characters.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/planets.csv -P /dataset
!ls /dataset

characters.csv	el_quijote.txt	frankenstein.txt  planets.csv


# RDD

---



## Example 1

In [7]:
textFile = spark.sparkContext.textFile("/dataset/frankenstein.txt")
textFile.first()

'FRANKENSTEIN'


### Creation of paralelized collection de colecciones paralelizadas
This is a fast way to create a RDD:

## Example 2

In [8]:
distData = spark.sparkContext.parallelize([25, 20, 15, 10, 5])
distData.reduce(lambda x ,y: x + y)

75

## Exercise 1
Count the number of lines for `el_quijote.txt` file

---



In [9]:
textfile = spark.sparkContext.textFile("/dataset/el_quijote.txt")
print("Number of lines: " + str(textFile.count()))

Number of lines: 7237


## Exercise 2
Print the first line of the file `el_quijote.txt`

---



In [11]:
textfile.first()

'DON QUIJOTE DE LA MANCHA'

## Transformations and Actions in RDDs 

### Actions

### Example 3

In [12]:
print(textFile.count()) # Number of elements in RDD
print(textFile.first()) # First element in RDD

7237
FRANKENSTEIN


### Transformaciones

### Example 4

In [13]:
# ReduceByKey
lines = spark.sparkContext.textFile("/dataset/frankenstein.txt")
pairs = lines.map(lambda s: (s, 1))
counts = pairs.reduceByKey(lambda a, b: a + b).cache()
counts.count()
counts.collect()

[('FRANKENSTEIN', 1),
 ('', 811),
 ('Letter 1', 1),
 ('commencement of an enterprise which you have regarded with such evil', 1),
 ('forebodings.  I arrived here yesterday, and my first task is to assure', 1),
 ('my dear sister of my welfare and increasing confidence in the success', 1),
 ('of my undertaking.', 1),
 ('feeling?  This breeze, which has travelled from the regions towards', 1),
 ('frost and desolation; it ever presents itself to my imagination as the', 1),
 ('region of beauty and delight.  There, Margaret, the sun is forever', 1),
 ('perpetual splendour.  There—for with your leave, my sister, I will put', 1),
 ('some trust in preceding navigators—there snow and frost are banished;', 1),
 ('solitudes.  What may not be expected in a country of eternal light?  I', 1),
 ('shall satiate my ardent curiosity with the sight of a part of the world',
  1),
 ('never before visited, and may tread a land never before imprinted by', 1),
 ('the foot of man. These are my enticements, and 

In [14]:
# SortByKey
sorted = counts.sortByKey()
sorted.collect()

[('', 811),
 ('                                                "Elizabeth Lavenza', 1),
 ('                                                August 26th, 17—', 1),
 ('                              "Alphonse Frankenstein.', 1),
 ('                    [Wordsworth\'s "Tintern Abbey".]', 1),
 ('               "Your affectionate and afflicted father,', 1),
 ('               Walton, in continuation.', 1),
 ('     An appetite; a feeling, and a love,', 1),
 ('     By thought supplied, or any interest', 1),
 ('     Haunted him like a passion:  the tall rock,', 1),
 ('     That had no need of a remoter charm,', 1),
 ('     The mountain, and the deep and gloomy wood,', 1),
 ('     Their colours and their forms, were then to him', 1),
 ("     Unborrow'd from the eye.", 1),
 ('     ——The sounding cataract', 1),
 ('    "Geneva, May 18th, 17—"', 1),
 ('   Embrace fond woe, or cast our cares away;', 1),
 ('   Nought may endure but mutability!', 1),
 ('   The path of its departure still is free.', 1),
 (

### Example 5

In [15]:
# Filter

linesWithSpark = textFile.filter(lambda line: "the" in line)
linesWithSpark.count()

3712

### Exercise 3
Get the word count for the file `frankenstein.txt`

---

In [16]:
words = spark.sparkContext.textFile("/dataset/frankenstein.txt")

words.flatMap(lambda x: x.split(" ")) \
.map(lambda s: (s, 1)) \
.reduceByKey(lambda a, b: a + b) \
.map(lambda x: (x[1], x[0])) \
.sortByKey(False) \
.collect()

[(3897, 'the'),
 (3488, ''),
 (2904, 'and'),
 (2720, 'I'),
 (2634, 'of'),
 (2072, 'to'),
 (1629, 'my'),
 (1338, 'a'),
 (1072, 'in'),
 (992, 'was'),
 (974, 'that'),
 (679, 'had'),
 (650, 'with'),
 (540, 'which'),
 (538, 'but'),
 (529, 'me'),
 (500, 'his'),
 (479, 'not'),
 (477, 'as'),
 (446, 'he'),
 (441, 'by'),
 (428, 'for'),
 (421, 'on'),
 (396, 'you'),
 (373, 'from'),
 (362, 'it'),
 (356, 'have'),
 (335, 'be'),
 (313, 'her'),
 (298, 'this'),
 (298, 'were'),
 (296, 'is'),
 (289, 'at'),
 (261, 'when'),
 (256, 'The'),
 (235, 'your'),
 (208, 'an'),
 (196, 'so'),
 (187, 'could'),
 (183, 'will'),
 (182, 'been'),
 (177, 'would'),
 (174, 'their'),
 (174, 'one'),
 (172, 'all'),
 (172, 'she'),
 (169, 'or'),
 (166, 'they'),
 (164, 'are'),
 (153, 'if'),
 (152, 'should'),
 (150, 'who'),
 (149, 'more'),
 (147, 'me,'),
 (147, 'him'),
 (146, 'no'),
 (136, 'some'),
 (130, 'these'),
 (130, 'now'),
 (128, 'But'),
 (126, 'He'),
 (124, 'into'),
 (123, 'upon'),
 (122, 'before'),
 (120, 'its'),
 (120, 'My'

### Exercise 4
Get TOP 10 of the words with more than 4 characters

---



In [17]:
words \
.flatMap(lambda line: line.split(" ")) \
.filter(lambda word: len(word) > 4) \
.map(lambda word: (word, 1)) \
.reduceByKey(lambda a, b: a + b) \
.map(lambda x: (x[1], x[0])) \
.sortByKey(False) \
.take(10)

[(540, 'which'),
 (187, 'could'),
 (177, 'would'),
 (174, 'their'),
 (152, 'should'),
 (130, 'these'),
 (122, 'before'),
 (107, 'might'),
 (105, 'myself'),
 (103, 'every')]

## Key/Value Pair RDD

---



### Example 6


---



In [18]:
charac_sw = spark.sparkContext.textFile("/dataset/characters.csv")
planets_sw = spark.sparkContext.textFile("/dataset/planets.csv")
charac_sw.take(10)

['name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,species',
 'Luke Skywalker,172,77,blond,fair,blue,19BBY,male,Tatooine,Human',
 'C-3PO,167,75,NA,gold,yellow,112BBY,NA,Tatooine,Droid',
 'R2-D2,96,32,NA,"white, blue",red,33BBY,NA,Naboo,Droid',
 'Darth Vader,202,136,none,white,yellow,41.9BBY,male,Tatooine,Human',
 'Leia Organa,150,49,brown,light,brown,19BBY,female,Alderaan,Human',
 'Owen Lars,178,120,"brown, grey",light,blue,52BBY,male,Tatooine,Human',
 'Beru Whitesun lars,165,75,brown,light,blue,47BBY,female,Tatooine,Human',
 'R5-D4,97,32,NA,"white, red",red,NA,NA,Tatooine,Droid',
 'Biggs Darklighter,183,84,black,light,brown,24BBY,male,Tatooine,Human']

In [19]:
planets_sw.take(10)

['name;rotation_period;orbital_period;diameter;climate;gravity;terrain;surface_water;population',
 'Alderaan;24;364;12500;temperate;1 standard;grasslands, mountains;40;2000000000',
 'Yavin IV;24;4818;10200;temperate, tropical;1 standard;jungle, rainforests;8;1000',
 'Hoth;23;549;7200;frozen;1.1 standard;tundra, ice caves, mountain ranges;100;NA',
 'Dagobah;23;341;8900;murky;N/A;swamp, jungles;8;NA',
 'Bespin;12;5110;118000;temperate;1.5 (surface), 1 standard (Cloud City);gas giant;0;6000000',
 'Endor;18;402;4900;temperate;0.85 standard;forests, mountains, lakes;8;30000000',
 'Naboo;26;312;12120;temperate;1 standard;grassy hills, swamps, forests, mountains;12;4500000000',
 'Coruscant;24;368;12240;temperate;1 standard;cityscape, mountains;NA;1E+12',
 'Kamino;27;463;19720;temperate;1 standard;ocean;100;1000000000']

In [20]:
from itertools import islice

charac_sw_noheader = charac_sw.mapPartitionsWithIndex(
    lambda idx, it: islice(it, 1, None) if idx == 0 else it)

planets_sw_noheader = planets_sw.mapPartitionsWithIndex(
    lambda idx, it: islice(it, 1, None) if idx == 0 else it)

### Exercise 5
Get a list of the population of the planet each Star Wars character belongs to

---


In [21]:
planets_sw_pair = planets_sw_noheader \
.map(lambda line: line.split(";")) \
.map(lambda x: (x[0], x[8]))


characters_sw_pair = charac_sw_noheader \
.map(lambda line: line.split(",")) \
.map(lambda x: (x[8], x[0]))

characters_sw_pair\
.join(planets_sw_pair)\
.map(lambda x: (x[0], x[1][1]))\
.distinct()\
.take(10)

[('Endor', '30000000'),
 ('Cerea', '450000000'),
 ('Iktotch', 'NA'),
 ('Quermia', 'NA'),
 ('Naboo', '4500000000'),
 ('Trandosha', '42000000'),
 ('Rodia', '1300000000'),
 ('Bespin', '6000000'),
 ('Champala', '3500000000'),
 ('Utapau', '95000000')]